**Note about deploye cloud composeer (managed airflow on gcp)**
- reference link for deployment: https://cloud.google.com/composer/docs/composer-2/terraform-create-environments#create_a_custom_service_account_in_your_project
- reference for use cloud composer to automate take vm snapshot: https://cloud.google.com/architecture/automating-infrastructure-using-cloud-composer
 

In [ ]:
#note about role binding (combine principle and role) to a service account

# Allow SA service account use the default GCE account
    #sa service account can use google_compute_default_service_account
resource "google_service_account_iam_member" "gce-default-account-iam" {
  service_account_id = data.google_compute_default_service_account.default.name
  role               = "roles/iam.serviceAccountUser"
  member             = "serviceAccount:${google_service_account.sa.email}"
}

**Labels and Tags in gcp**
- Tags is used for fine-grained access control on group of resource
- Labels is metadata for the resource which help in manage and analytics of resource (cost or usage of each team)
- reference about labels: https://cloud.google.com/compute/docs/labeling-resources
- referecne about tags: https://cloud.google.com/resource-manager/docs/tags/tags-overview

**variables.tf vs terraform.tfvar**
- variables.tf: is used to define vaiables type and optional default value
- terraform.tfvar (*.tfvar): is assing values to varialbe

In [ ]:
gcloud composer environments run ENVIRONMENT \
  --location LOCATION \
  variables -- \
  --set project_id PROJECT_ID

In [ ]:
#descibe your cloud composer
#environments in cloud composer object itself
gcloud composer environments describe composer-env --location=$region

In [ ]:
#add dag to composer
#error on multiple line notic
gcloud composer environments storage dags import \
    --environment=$composer_env \
    --location=$region \
    --source="$dag_one"

In [ ]:
#reference about airflow variable: https://airflow.apache.org/docs/apache-airflow/stable/concepts/variables.html
#reference about airflow cli reference: https://airflow.apache.org/docs/apache-airflow/stable/cli-and-env-variables-ref.html
# set variable with gcloud command
gcloud composer environments run $composer_env --location=$region \
    variables -- \
    set bucket_path $bucket_path
gcloud composer environments run $composer_env --location=$region \
    variables -- \
    set gce_zone $gce_zone
gcloud composer environments run $composer_env --location=$region \
    variables -- \
    set project_id $project_id


**Note about config cloud composer with terraform**:
- reference link: https://cloud.google.com/composer/docs/composer-2/terraform-create-environments#agent-permissions
- to deploy cloud composer in gcp we need:
    - google_project_service: to enable cloud composer service and api
    - google_service_account: to create service account for composer worker machine
    - google_project_iam_member: to role binding (justed created sa and roles/composer.worker) at project level
        - this verb the resource is project, we allow service account have role (a set of permission) on this project
    -google_service_account_iam_member: to allow default composer agent to use sa workder.
        - resource is composer worker service account
        - role-binding:
            -member: default commposer agent sa
            -roles: roles/composer.ServiceAgentV2Ext
            -purpose: allow member (default composer agent sa) to use composer worker sa
    google_composer_environment: to deploy cloud composer environment in gcp
        -we have two simple config:
            -image
            -service_account for node

In [ ]:
**Note about launching dataflow pipeline with cloud composer**
- setting up before deploy:
    - create empty bigquery table with schema definition
        - using terraform: https://registry.terraform.io/providers/hashicorp/google/latest/docs/resources/bigquery_table
        - reference for heredoc string in terraform: https://developer.hashicorp.com/terraform/language/expressions/strings#heredoc-strings
        - reference about file function: https://developer.hashicorp.com/terraform/language/functions/file. this function return string
    - create a storage bucket:
        - to have storage bucket permission control (iam) use gsutil iam command: 
        - bucket access control with t